In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pandas as pd
path='annotations.csv'
relative_path='train/train/'
df=pd.read_csv(path)
data=[]
df.head()

,Unnamed: 0,fname,label,split
0,0,02639.npy,Bark,train
1,1,1580.npy,Bark,train
2,2,24030.npy,Bark,train
3,3,316499.npy,Bark,train
4,4,413718.npy,Bark,train


In [2]:
for x in df['fname']:
    address=relative_path+x
    data.append(np.load(address))

In [3]:
label=df['label']
label

0                    Bark
1                    Bark
2                    Bark
3                    Bark
4                    Bark
              ...        
995    Walk_and_footsteps
996    Walk_and_footsteps
997    Walk_and_footsteps
998    Walk_and_footsteps
999    Walk_and_footsteps
Name: label, Length: 1000, dtype: object

In [4]:
mx_dim=0
mn_dim=10000
for sz in data:
    mx_dim=max(mx_dim,sz.shape[2])
    mn_dim=min(mn_dim,sz.shape[2])
mx_dim,mn_dim


(2584, 29)

In [5]:
final_data=[]
for spectrogram in data:
    x=spectrogram.flatten()
    y=np.zeros((mx_dim-spectrogram.shape[2])*128)
    x=np.append(x,y)
    final_data.append(x)
    

In [6]:
final_data=np.array(final_data)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
final_data=scaler.fit_transform(final_data)


In [14]:
from sklearn.preprocessing import OrdinalEncoder
label=np.array(label)
label=label.reshape(-1,1)
oc=OrdinalEncoder()
label=oc.fit_transform(label)


In [15]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(final_data,label,test_size=0.2,random_state=42)

In [16]:
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression(max_iter=30)
log_reg_clf=log_reg.fit(train_x,train_y)

C:\Users\DELL\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
print(log_reg_clf.score(test_x,test_y))
train_y_pred=log_reg.predict(train_x)

0.625


In [18]:
from sklearn.metrics import confusion_matrix
confusion_log_clf=confusion_matrix(train_y_pred,train_y)

In [19]:
p=(train_y_pred==train_y).sum()
p,train_y.shape

(64221, (800, 1))

In [20]:
from sklearn.metrics import f1_score
f1_score(train_y_pred,train_y,average='weighted')

0.9900172560319246

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
import tensorflow as tf
model1 = Sequential([
    Dense(64, activation='tanh', input_shape = train_x[0].shape),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='softmax')
])
print(model1.summary())
model1.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['acc'])

hist = model1.fit(train_x, train_y, epochs=10, batch_size=64)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 64)                21168192  
                                                                 
 dense_29 (Dense)            (None, 32)                2080      
                                                                 
 dense_30 (Dense)            (None, 16)                528       
                                                                 
 dense_31 (Dense)            (None, 8)                 136       
                                                                 
 dense_32 (Dense)            (None, 1)                 9         
                                                                 
Total params: 21,170,945
Trainable params: 21,170,945
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
13/13 [===================

In [41]:
train_y_pred_softmax=model1.predict(train_x)


25/25 [==============================] - 26s 1s/step


In [42]:
correct=(train_y_pred_softmax==train_y).sum()
correct

0

In [43]:
print(model1.evaluate(test_x,test_y))

7/7 [==============================] - 17s 109ms/step - loss: nan - acc: 0.1000
[nan, 0.10000000149011612]
